# 1 Housekeeping

## 1.1 Load libraries

In [3]:
import pandas as pd
import numpy as np

In [4]:
pd.set_option("display.max_columns", 100)
pd.set_option("display.max_rows", 100)

## 1.2 Load data

#### Thefts

In [27]:
df = pd.read_csv(r'D:\Jupyter Notebooks\cbs-diefstal\data\cbs_diefstallen.csv', sep = ';', na_values ='       .')
df = df[df['GebruikVanGeweld']== 'T001540']
df = df.drop('GebruikVanGeweld', axis=1)
df = df[df['SoortDiefstal']!= 'CRI1134' ]
df['Perioden'] = df['Perioden'].str.split('JJ00').str[0].astype('int')

df = df.rename(columns = {'TotaalGeregistreerdeDiefstallen_1': 'count', 'GeregistreerdeDiefstallenPer1000Inw_3' : 'Rel_diefstal',
                         'Perioden': 'year'})
df = df.drop('Rel_diefstal', axis=1)

In [40]:
df['SoortDiefstal'] = np.where(df['SoortDiefstal']== 'CRI1122', 'bike theft', df['SoortDiefstal']  )
df['SoortDiefstal'] = np.where(df['SoortDiefstal']== 'CRI1123', 'moped theft', df['SoortDiefstal']  )

df['SoortDiefstal'] = np.where(df['SoortDiefstal']== 'CRI1124', 'motorcycle/scooter theft', df['SoortDiefstal']  )
df['SoortDiefstal'] = np.where(df['SoortDiefstal']== 'CRI1125', 'car theft', df['SoortDiefstal']  )

df['SoortDiefstal'] = np.where(df['SoortDiefstal']== 'CRI1127', 'boat theft', df['SoortDiefstal']  )
#df['SoortDiefstal'] = np.where(df['SoortDiefstal']== 'CRI1128', 'Diefstal uit/vanaf personenauto', df['SoortDiefstal']  )

df['SoortDiefstal'] = np.where(df['SoortDiefstal']== 'CRI1131', 'animal theft', df['SoortDiefstal']  )
df['SoortDiefstal'] = np.where(df['SoortDiefstal']== 'CRI1132', 'street robbery', df['SoortDiefstal']  )

df['SoortDiefstal'] = np.where(df['SoortDiefstal']== 'CRI1133', 'pickpocketing', df['SoortDiefstal']  )
df['SoortDiefstal'] = np.where(df['SoortDiefstal']== 'CRI1135', 'home burglary', df['SoortDiefstal'] )

df['SoortDiefstal'] = np.where(df['SoortDiefstal']== 'CRI1137', 'shoplifting', df['SoortDiefstal']  )
df['SoortDiefstal'] = np.where(df['SoortDiefstal']== 'CRI1144', 'heist', df['SoortDiefstal'] )

df = df[df['SoortDiefstal']!= 'CRI1128']
df.head()

,ID,SoortDiefstal,RegioS,year,count
0,6116,bike theft,GM1680,2018,25.0
1,6117,bike theft,GM1680,2019,50.0
2,6118,bike theft,GM1680,2020,20.0
3,6119,bike theft,GM1680,2021,30.0
4,6128,bike theft,GM0738,2018,5.0


#### Municipalities

In [41]:
gemeentes_2018= pd.read_csv(r'D:\Jupyter Notebooks\cbs-diefstal\data\gemeente_2018.csv')
gemeentes_2019= pd.read_csv(r'D:\Jupyter Notebooks\cbs-diefstal\data\gemeente_2019.csv')
gemeentes_2020= pd.read_csv(r'D:\Jupyter Notebooks\cbs-diefstal\data\gemeente_2020.csv')
gemeentes_2021= pd.read_csv(r'D:\Jupyter Notebooks\cbs-diefstal\data\gemeente_2021.csv')

#merge
gemeentes = gemeentes_2018.append(gemeentes_2019).append(gemeentes_2020).append(gemeentes_2021)
gemeentes = gemeentes.drop_duplicates(subset=['Gemeentecode', 'GemeentecodeGM', 'Gemeentenaam'], keep='last')
gemeentes.head()

C:\Users\Brenda\AppData\Local\Temp\ipykernel_1312\38684022.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  gemeentes = gemeentes_2018.append(gemeentes_2019).append(gemeentes_2020).append(gemeentes_2021)
C:\Users\Brenda\AppData\Local\Temp\ipykernel_1312\38684022.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  gemeentes = gemeentes_2018.append(gemeentes_2019).append(gemeentes_2020).append(gemeentes_2021)


,Gemeentecode,GemeentecodeGM,Gemeentenaam,Provinciecode,ProvinciecodePV,Provincienaam
1,738,GM0738,Aalburg,30,PV30,Noord-Brabant
25,5,GM0005,Bedum,20,PV20,Groningen
40,585,GM0585,Binnenmaas,28,PV28,Zuid-Holland
46,9,GM0009,Ten Boer,20,PV20,Groningen
64,611,GM0611,Cromstrijen,28,PV28,Zuid-Holland


#### Geojson file

In [42]:
#html_link
gemeente_json_html = f"https://gis.vng.nl/v2/assets/geojson/g2020.geojson" 

#geojson
gemeente_json = geojson.load(open('geojson_gemeente_2020.geojson', 'r'))
gemeente_json ['features'][1]


{"geometry": {"coordinates": [[[4.14489, 51.931409], [4.135763, 51.906156], [4.155279, 51.881603], [4.117245, 51.857958], [4.067286, 51.845403], [4.030059, 51.884463], [4.05306, 51.914742], [4.038961, 51.923734], [3.992016, 51.915495], [4.065845, 51.932619], [4.14489, 51.931409]]], "type": "Polygon"}, "properties": {"code": "0614", "fid": 2, "gemeentena": "Westvoorne"}, "type": "Feature"}

In [43]:
#create id in geojson
for feature in gemeente_json['features']:
    feature['id'] = feature['properties']['code']

# 2 Pre-process

### 2.1 Merge datasets

In [44]:
data = df.merge(gemeentes[['GemeentecodeGM', 'Gemeentenaam', 'Provincienaam']], how = 'inner', left_on = 'RegioS', right_on = 'GemeentecodeGM')
data = data[['ID', 'Provincienaam', 'GemeentecodeGM',  'Gemeentenaam', 'SoortDiefstal',  'year', 'count']]
data.columns = [x.lower() for x in data.columns]
data = data.rename(columns = {'provincienaam': 'province', 'gemeentecodegm' : 'municipality_id', 
                                         'gemeentenaam': 'municipality', 'soortdiefstal': 'theft'})
data.head()
data.to_csv(r'D:\Jupyter Notebooks\cbs-diefstal\data\data.csv', index=False)

### 2.2 pivot datasets

In [55]:
#create pivot
data_pivot = pd.pivot_table(data.drop('id', axis=1) , index = ['province', 'municipality_id',  'municipality', 'year'], 
                            columns = 'theft',
                           aggfunc = np.mean,
                           fill_value = np.nan)

#create list of multi-level column names
mi = data_pivot.columns
mi.tolist()


#only keep the second element in the list
ind = pd.Index( [e[0] + "_" + e[1] for e in mi.tolist()])

#set the list as the column title
data_pivot.columns = ind

#flatten multlevel index
data_pivot = data_pivot.rename_axis(['province', 'municipality_id',  'municipality', 'year']).reset_index()

#create id
data_pivot['id'] = data_pivot['municipality_id'].str.split('GM').str[1]
data_pivot['id'] = data_pivot['id'].astype('str')
data_pivot = data_pivot.drop('municipality_id', axis=1)

#rename
data_pivot = data_pivot.rename(columns = {'count_bike theft' : 'bike theft', 'count_animal theft': 'animal theft',
                                         'count_boat theft': 'boat theft', 'count_car' : 'car theft', 
                                         'count_heist': 'heist', 'count_moped theft' : 'moped theft',
                                         'count_motorcycle/scooter theft': 'motorcycle/scooter theft',
                                         'count_pickpocketing' :'pickpocketing', 'count_shoplifting' : 'shoplifting',
                                         'count_street robbery':'street robbery'})

#re-order
data_pivot = data_pivot[['id', 'province', 'municipality', 'year', 'animal theft', 'bike theft',
       'boat theft', 'car theft',  'moped theft', 'motorcycle/scooter theft',  'heist',
       'pickpocketing', 'shoplifting','street robbery']]

data_pivot.head()
data_pivot.to_csv(r'D:\Jupyter Notebooks\cbs-diefstal\data\data_pivot.csv', index=False)